In [52]:
from langchain.agents import initialize_agent, AgentType
from langchain.chains import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [53]:
# load environment variables 
import os
from dotenv import dotenv_values, load_dotenv
load_dotenv()
openai_api_key = os.environ['OPENAI_API_KEY']

In [54]:
model = "gpt-3.5-turbo"
temperature = 0
llm = ChatOpenAI(model=model, temperature=temperature, openai_api_key=openai_api_key)

In [55]:
from backend.triage import triage_biopsy
triage_biopsy(1.5, True, 1.2, False, 0.5, True)

(Core(n='first', length=1.5, cortex=True),
 'Assign the first core that is 1.5 cm long as the diagnostic core. If the amount of cortext is less than 0.5 cm then please the entire core in formalin for light microscopy. Otherwise, divide diagnostic core in three parts with a minimum of 0.1-0.2 cm of cortex fixed in 2.5% glutaraldehyde for EM, 0.3 cm cortex frozen in OCT for IF, and the remaining amount of tissue fixed in 10% formalin for LM. \nThe second biopsy core should be embedded in OCT, frozen on dry ice and shipped to the central bio-repository where it will be distributed to the TISs.')

In [56]:
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

class TriageInput(BaseModel):
    core1_length: float = Field()
    core1_cortex: bool = Field()
    core2_length: float = Field()
    core2_cortex: bool = Field()
    core3_length: float = Field()
    core3_cortex: bool = Field()

In [57]:
tools = [
    StructuredTool(
        name = "triage_biopsy",
        func = triage_biopsy,
        description = "Use this tool to triage a kidney biopsy. It produces instructions to process the biopsy bases on the length of the cores and whether renal or kidney cortext is visualized in them. If cortex is not mentioned in the text, then assume that it was not visualized",
        args_schema = TriageInput
    )
]

In [58]:
# AgentType = STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION | OPENAI_FUNCTIONS | OPENAI_MULTI_FUNCTIONS
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [51]:
agent.run("You obtain three cores, first core is 1.5 cm in length, second is 1 cm, and thrid is 0.5 cm. You see cortex in first core. How would you triage the tissue?")



> Entering new AgentExecutor chain...

Invoking: `triage_biopsy` with `{'core1_length': 1.5, 'core1_cortex': True, 'core2_length': 1, 'core2_cortex': False, 'core3_length': 0.5, 'core3_cortex': False}`


(Core(n='first', length=1.5, cortex=True), 'Assign the first core that is 1.5 cm long as the diagnostic core. If the amount of cortext is less than 0.5 cm then please the entire core in formalin for light microscopy. Otherwise, divide diagnostic core in three parts with a minimum of 0.1-0.2 cm of cortex fixed in 2.5% glutaraldehyde for EM, 0.3 cm cortex frozen in OCT for IF, and the remaining amount of tissue fixed in 10% formalin for LM. \n')Based on the information provided, here is how you should triage the tissue:

1. Assign the first core, which is 1.5 cm in length and shows cortex, as the diagnostic core.
2. If the amount of cortex in the diagnostic core is less than 0.5 cm, please place the entire core in formalin for light microscopy.
3. If the amount of cortex in the diagnos

'Based on the information provided, here is how you should triage the tissue:\n\n1. Assign the first core, which is 1.5 cm in length and shows cortex, as the diagnostic core.\n2. If the amount of cortex in the diagnostic core is less than 0.5 cm, please place the entire core in formalin for light microscopy.\n3. If the amount of cortex in the diagnostic core is 0.5 cm or more, divide the diagnostic core into three parts:\n   - Take a minimum of 0.1-0.2 cm of cortex and fix it in 2.5% glutaraldehyde for electron microscopy.\n   - Take 0.3 cm of cortex and freeze it in OCT for immunofluorescence.\n   - Fix the remaining amount of tissue in 10% formalin for light microscopy.\n\nPlease follow these instructions to process the biopsy.'

In [7]:
agent.run("You obtain three cores, first core is 1.5 cm in length, second is 1 cm, and third is 0.5 cm. Cortex was not visualized. How would you triage the tissue?")

NameError: name 'agent' is not defined

In [43]:
agent.run("You obtain three cores, first core is 1.2 cm in length, second is 1 cm, and third is 0.5 cm. Cortex was not visualized. How would you triage the tissue?")



> Entering new AgentExecutor chain...

Invoking: `triage_biopsy` with `{'core1_length': 1.2, 'core1_cortex': False, 'core2_length': 1, 'core2_cortex': False, 'core3_length': 0.5, 'core3_cortex': False, 'n_passes': 3}`


All biopsy cores are less than 1.4 cm. A minimum of 0.5 cm of cortex needs to be placed into formalin for LM. A minimum of 0.2 cm cortex needs to be frozen in OCT to be processed for IF. A minimum of 0.1 cm cortex needs to be placed glutaraldehyde for EM. In the event of scarce tissue availability, prioritize LM > IF > EM. 
The remaining two cores will be designated for tissue interrogation and triaged/processed as cores #2 and #3.

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


Based on the information provided, here is how you should triage the tissue:

1. Core 1: Since the length is 1.2 cm and cortex was not visualized, this core should be designated for tissue interrogation and triaged/processed as Core #1.

2. Core 2: Since the length is 1 cm and cortex was not visualized, this core should be designated for tissue interrogation and triaged/processed as Core #2.

3. Core 3: Since the length is 0.5 cm and cortex was not visualized, this core should be designated for tissue interrogation and triaged/processed as Core #3.

Please note that in the event of scarce tissue availability, prioritize LM (light microscopy) over IF (immunofluorescence) and EM (electron microscopy).

> Finished chain.


'Based on the information provided, here is how you should triage the tissue:\n\n1. Core 1: Since the length is 1.2 cm and cortex was not visualized, this core should be designated for tissue interrogation and triaged/processed as Core #1.\n\n2. Core 2: Since the length is 1 cm and cortex was not visualized, this core should be designated for tissue interrogation and triaged/processed as Core #2.\n\n3. Core 3: Since the length is 0.5 cm and cortex was not visualized, this core should be designated for tissue interrogation and triaged/processed as Core #3.\n\nPlease note that in the event of scarce tissue availability, prioritize LM (light microscopy) over IF (immunofluorescence) and EM (electron microscopy).'

In [9]:
agent.run("You do one pass, and obtain one core that's 1.5 cm long. How would you triage the tissue?")



> Entering new AgentExecutor chain...

Invoking: `triage_biopsy` with `{'core1_length': 1.5, 'core1_cortex': False, 'core2_length': 0, 'core2_cortex': False, 'core3_length': 0, 'core3_cortex': False, 'n_passes': 1}`


(Core(n='first', length=1.5, cortex=False), 'The entire tissue will be fixed in formalin to be processed for Light Microscopy.')Based on the information provided, the triage for the tissue would be as follows:

- The entire tissue will be fixed in formalin to be processed for Light Microscopy.

> Finished chain.


'Based on the information provided, the triage for the tissue would be as follows:\n\n- The entire tissue will be fixed in formalin to be processed for Light Microscopy.'

In [59]:
agent.run("I finished the biopsy. I only have one core which is 1.2 cm in length. How should I triage the tissue?")



> Entering new AgentExecutor chain...

Invoking: `triage_biopsy` with `{'core1_length': 1.2, 'core1_cortex': False, 'core2_length': 0, 'core2_cortex': False, 'core3_length': 0, 'core3_cortex': False}`


(Core(n='first', length=1.2, cortex=False), 'The entire tissue will be fixed in formalin to be processed for Light Microscopy.')Based on the information provided, since you only have one core which is 1.2 cm in length and the cortex was not visualized, the entire tissue should be fixed in formalin to be processed for Light Microscopy.

> Finished chain.


'Based on the information provided, since you only have one core which is 1.2 cm in length and the cortex was not visualized, the entire tissue should be fixed in formalin to be processed for Light Microscopy.'

In [60]:
agent.run("I finished the biopsy. I only 3 cores. One is 1.5 cm in length, the second is 1.2 cm, and the third is 0.8 cm in length. Cortex was visualized in all of them. How should I triage the tissue?")




> Entering new AgentExecutor chain...

Invoking: `triage_biopsy` with `{'core1_length': 1.5, 'core1_cortex': True, 'core2_length': 1.2, 'core2_cortex': True, 'core3_length': 0.8, 'core3_cortex': True}`


(Core(n='first', length=1.5, cortex=True), 'Assign the first core that is 1.5 cm long as the diagnostic core. If the amount of cortext is less than 0.5 cm then please the entire core in formalin for light microscopy. Otherwise, divide diagnostic core in three parts with a minimum of 0.1-0.2 cm of cortex fixed in 2.5% glutaraldehyde for EM, 0.3 cm cortex frozen in OCT for IF, and the remaining amount of tissue fixed in 10% formalin for LM. \nThe remaining two cores will be designated for tissue interrogation and triaged/processed as cores #2 and #3.')Based on the length of the cores and the visualization of cortex in all of them, here is how you should triage the tissue:

1. Assign the first core that is 1.5 cm long as the diagnostic core.
   - If the amount of cortex is less than 0.5

'Based on the length of the cores and the visualization of cortex in all of them, here is how you should triage the tissue:\n\n1. Assign the first core that is 1.5 cm long as the diagnostic core.\n   - If the amount of cortex is less than 0.5 cm, please place the entire core in formalin for light microscopy.\n   - If the amount of cortex is 0.5 cm or more, divide the diagnostic core into three parts:\n     - Fix a minimum of 0.1-0.2 cm of cortex in 2.5% glutaraldehyde for electron microscopy.\n     - Freeze 0.3 cm of cortex in OCT for immunofluorescence.\n     - Fix the remaining amount of tissue in 10% formalin for light microscopy.\n\n2. The remaining two cores will be designated for tissue interrogation and triaged/processed as cores #2 and #3.\n\nPlease follow these instructions to process the biopsy tissue accordingly.'

In [61]:
agent.run("core one is 1.5 cm, core two is 1.2 cm, and core 3 is 0.8 cm. Each core is 50% cortex")



> Entering new AgentExecutor chain...

Invoking: `triage_biopsy` with `{'core1_length': 1.5, 'core1_cortex': True, 'core2_length': 1.2, 'core2_cortex': True, 'core3_length': 0.8, 'core3_cortex': True}`


(Core(n='first', length=1.5, cortex=True), 'Assign the first core that is 1.5 cm long as the diagnostic core. If the amount of cortext is less than 0.5 cm then please the entire core in formalin for light microscopy. Otherwise, divide diagnostic core in three parts with a minimum of 0.1-0.2 cm of cortex fixed in 2.5% glutaraldehyde for EM, 0.3 cm cortex frozen in OCT for IF, and the remaining amount of tissue fixed in 10% formalin for LM. \nThe remaining two cores will be designated for tissue interrogation and triaged/processed as cores #2 and #3.')

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


Based on the information provided, here are the instructions to process the kidney biopsy:

1. Assign the first core, which is 1.5 cm long and contains cortex, as the diagnostic core.
2. If the amount of cortex in the diagnostic core is less than 0.5 cm, please place the entire core in formalin for light microscopy.
3. If the amount of cortex in the diagnostic core is 0.5 cm or more, divide the diagnostic core into three parts:
   - Fix a minimum of 0.1-0.2 cm of cortex in 2.5% glutaraldehyde for electron microscopy.
   - Freeze 0.3 cm of cortex in OCT for immunofluorescence.
   - Fix the remaining amount of tissue in 10% formalin for light microscopy.
4. The remaining two cores will be designated for tissue interrogation and should be triaged/processed as cores #2 and #3.

Please follow these instructions to process the biopsy accordingly.

> Finished chain.


'Based on the information provided, here are the instructions to process the kidney biopsy:\n\n1. Assign the first core, which is 1.5 cm long and contains cortex, as the diagnostic core.\n2. If the amount of cortex in the diagnostic core is less than 0.5 cm, please place the entire core in formalin for light microscopy.\n3. If the amount of cortex in the diagnostic core is 0.5 cm or more, divide the diagnostic core into three parts:\n   - Fix a minimum of 0.1-0.2 cm of cortex in 2.5% glutaraldehyde for electron microscopy.\n   - Freeze 0.3 cm of cortex in OCT for immunofluorescence.\n   - Fix the remaining amount of tissue in 10% formalin for light microscopy.\n4. The remaining two cores will be designated for tissue interrogation and should be triaged/processed as cores #2 and #3.\n\nPlease follow these instructions to process the biopsy accordingly.'